In [185]:
%load_ext autoreload
%autoreload 2
import tensorflow_quantum as tfq
from utilities.variational import VQE
from utilities.qmodels import *
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.unitary_killer import UnitaryMurder
from utilities.simplifier import Simplifier
import cirq
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [215]:

vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=40, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits,testing=True)

#indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
indexed_circuit = [5,4,2,4,3,5,3,0,5,5]
#indexed_circuit = [5,3,3,5,4,0,5,5,3,3,5,3,5]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
energy, resolver, history = vqe_handler.vqe(indexed_circuit)

model = QNN(symbols=list(resolver.keys()), observable=vqe_handler.observable)
print(model(cirq.resolve_parameters(circuit,resolver)), energy)


tf.Tensor([-7.999999], shape=(1,), dtype=float32) tf.Tensor(-7.999997, shape=(), dtype=float32)


In [216]:
#Mindexed_circuit, Mresolver, Mindex_to_symbols = iid_inserter.place_identities(indexed_circuit,symbol_to_value=resolver,rate_iids_per_step=2)
Mcircuit = vqe_handler.give_unitary(indexed_circuit, resolver)
Mcircuit

(0, 0): ───Rx(-0.5π)───Rz(0.5π)──────Rx(0.199π)─────────────────@──────────────────────────────
                                                                │
(0, 1): ───Rx(1.12π)───Rz(-0.679π)───Rx(-1.9π)────Rz(-0.606π)───X───Rx(-1.19π)───Rx(-0.703π)───

In [217]:
i,r,po=simplifier.simplify_step(indexed_circuit,resolver,index_to_symbols)

5
qubit 1 symdel ['th_6']
4
here:  ['th_8']
-5.9452085 th_8


In [220]:
nu=vqe_handler.give_unitary(i,r)
nu

(0, 0): ───Rx(-0.5π)───Rz(0.5π)────Rx(0.199π)───@─────────────────
                                                │
(0, 1): ───Rz(-1.7π)───Rx(1.11π)───Rz(-π)───────X───Rx(-0.679π)───

In [219]:
model(nu)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.7012386], dtype=float32)>

In [234]:
vals = np.array([1.12, -.679, -1.9, -.606])*np.pi

In [235]:
inde = [5,3,5,3]
c, symbols, posi = vqe_handler.give_circuit(inde)
res = {r:m for r,m in zip(symbols,vals)}

In [236]:
ii, rr, pp= simplifier.simplify_step(inde, res, posi)

5
qubit 1 symdel ['th_3']


In [242]:
model(vqe_handler.give_unitary(ii,rr))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-3.724611], dtype=float32)>

In [250]:
co = vqe_handler.give_unitary(inde,res)
co.append(cirq.rx(0).on(vqe_handler.qubits[0]))
model(co)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-3.7246099], dtype=float32)>

In [192]:
vqe_handler.give_circuit(indexed_circuit)[0]

(0, 0): ───Rx(th_1)───Rz(th_2)───Rx(th_3)──────────────@───Rz(th_13)─────────────────────────────────────────────────────────────
                                                       │
(0, 1): ───Rx(th_0)───Rz(th_4)───Rx(th_5)───Rz(th_6)───X───Rx(th_7)────Rx(th_8)───Rx(th_9)───Rx(th_10)───Rx(th_11)───Rx(th_12)───

In [197]:
np.sum(list(resolver.values())[7:])

37.32575773325202

In [182]:

vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=40, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits,testing=True)

#indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
indexed_circuit = [5,5,5,1,5,5,5]
#indexed_circuit = [5,3,3,5,4,0,5,5,3,3,5,3,5]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
resolver = {s:r for s,r in zip(symbols, np.random.randn(len(symbols)))}
#energy, resolver, history = vqe_handler.vqe(indexed_circuit)

model = QNN(symbols=list(resolver.keys()), observable=vqe_handler.observable)
print(model(cirq.resolve_parameters(circuit,resolver)))


tf.Tensor([-1.035732], shape=(1,), dtype=float32)


In [183]:
i,r,po=simplifier.simplify_step(indexed_circuit,resolver,index_to_symbols)

4
here:  ['th_1', 'th_2']
0.8779985591597353 th_0
4
here:  ['th_4', 'th_5']
1.9401020044321542 th_1


In [184]:
nu=vqe_handler.give_unitary(i,r)
nu
model(nu)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-1.0357327], dtype=float32)>